In [1]:
import sys
sys.path.append('../')
import exp
import regression as r

In [2]:
df = exp.get_wp_energy_data_landsat()
df.head()

,PET,VPD,air_temp,doy,precip,soil_temp,sw_in,wind_speed,year,wp_RNET,...,wp_h,wp_le,wp_evi,wp_lswi2,wp_ndvi,wp_LST.day,wp_LST.night,LW_IN.wp,LW_IN.si,LW_IN
0,7.33,0.808731,19.179167,195,0.0,22.320833,30.3156,4.958333,2012,20.798342,...,2.605590,13.643902,0.377924,0.413754,0.594708,31.567899,17.204530,29.105844,24.562860,29.105844
1,6.52,0.755945,19.325000,196,0.0,21.770833,29.6316,3.791667,2012,20.573593,...,3.459300,13.498761,0.381184,0.420529,0.597960,29.570000,17.390000,29.760736,26.096551,29.760736
2,6.92,0.858993,20.262500,197,0.0,21.908333,29.3472,4.137500,2012,20.475931,...,2.117474,14.987823,0.384444,0.427304,0.601212,31.097908,17.235624,30.299442,25.986418,30.299442
3,6.35,0.477617,16.791667,198,0.0,22.420833,28.8180,6.033333,2012,20.571045,...,4.636145,11.355851,0.387704,0.434079,0.604465,30.868718,17.248525,28.978868,25.122209,28.978868
4,5.13,0.556820,17.016667,199,0.0,21.529167,23.1732,4.350000,2012,16.757401,...,3.448946,10.943102,0.390963,0.440854,0.607717,30.657792,17.259663,30.653667,27.830367,30.653667


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1028 entries, 0 to 1027
Data columns (total 24 columns):
PET             1028 non-null float64
VPD             1028 non-null float64
air_temp        1028 non-null float64
doy             1028 non-null int64
precip          1028 non-null float64
soil_temp       1028 non-null float64
sw_in           1028 non-null float64
wind_speed      1028 non-null float64
year            1028 non-null int64
wp_RNET         1028 non-null float64
wp_ch4_gf       1028 non-null float64
wp_co2_gf       1028 non-null float64
wp_er           1028 non-null float64
wp_gpp          1028 non-null float64
wp_h            1028 non-null float64
wp_le           1028 non-null float64
wp_evi          1028 non-null float64
wp_lswi2        1028 non-null float64
wp_ndvi         1028 non-null float64
wp_LST.day      1028 non-null float64
wp_LST.night    1028 non-null float64
LW_IN.wp        1028 non-null float64
LW_IN.si        1028 non-null float64
LW_IN           1028 no

In [4]:
train_cols = ["precip", "sw_in", "LW_IN.wp", "VPD", "air_temp", "PET", "wind_speed", "wp_LST.day",
             "wp_LST.night", "wp_evi", "wp_ndvi", "wp_lswi2"]
X, Y = exp.featurize(df, train_cols, ["wp_er"])
X, Y, scaler = r.preprocess(X, Y)
X.shape

(1028, 12)

In [5]:
r.random_forests_cross_val(X, Y, feature_names=train_cols)

Running Random Forests Cross Validation...
10-fold CV Acc Mean:  0.935721151453
CV Scores:  0.928589844726, 0.946825639362, 0.931279220233, 0.937982178856, 0.91568966844, 0.946056353392, 0.938838782406, 0.938354167222, 0.920589903958, 0.953005755936
OOB score: 0.937180456936
Feature Importances:
('wp_LST.night', 0.20611886057146969)
('wp_LST.day', 0.16644913764148139)
('air_temp', 0.15028666941848912)
('PET', 0.12175735432886928)
('sw_in', 0.08729565801550894)
('wind_speed', 0.06432669269573861)
('wp_evi', 0.057766128362992129)
('wp_lswi2', 0.055109298068824926)
('VPD', 0.038299497352787329)
('wp_ndvi', 0.037097137468410299)
('LW_IN.wp', 0.013998572615116863)
('precip', 0.0014949934603113682)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='sqrt', max_leaf_nodes=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=200, n_jobs=1, oob_score=True, random_state=None,
           verbose=0, warm_start=False)

In [6]:
r.xgb_trees_cross_val(X, Y, feature_names=train_cols)

Running Gradient Boosted Trees Cross Validation...
10-fold CV Acc Mean:  0.936600864852
CV Scores:  0.930109536274, 0.954192433251, 0.92922330252, 0.949586981738, 0.922530865005, 0.9322331528, 0.926071119263, 0.933354530955, 0.9286879626, 0.960018764109
Feature Importances:
('wp_ndvi', 0.14390786948940559)
('wp_evi', 0.12922565765540317)
('wp_lswi2', 0.12076455424553816)
('wp_LST.night', 0.11628248219563568)
('wp_LST.day', 0.096106569013111265)
('sw_in', 0.082607552254822331)
('wind_speed', 0.076336866397164554)
('air_temp', 0.064380280597828493)
('VPD', 0.05770102986379376)
('PET', 0.057550262915276831)
('LW_IN.wp', 0.043277116293755587)
('precip', 0.011859759078264533)


GradientBoostingRegressor(alpha=0.9, init=None, learning_rate=0.1, loss='ls',
             max_depth=3, max_features='sqrt', max_leaf_nodes=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=200,
             presort='auto', random_state=None, subsample=1.0, verbose=0,
             warm_start=False)

In [7]:
r.svc_cross_val(X, Y)

Running SVC Cross Validation...
10-fold CV Acc Mean:  0.854068277609
CV Scores:  0.841723168467, 0.866139349342, 0.842942596953, 0.842640448253, 0.824246955149, 0.873985041575, 0.856499705143, 0.878723888713, 0.862745028747, 0.851036593749


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [8]:
r.dnn_cross_val(X, Y)

Running Neural Network Cross Validation...
Step #1, avg. loss: 866.32080
Step #501, epoch #50, avg. loss: 22.50697
Step #1001, epoch #100, avg. loss: 6.64607
Step #1501, epoch #150, avg. loss: 4.65791
Step #2001, epoch #200, avg. loss: 3.35010
Step #2501, epoch #250, avg. loss: 2.64225
Step #3001, epoch #300, avg. loss: 2.23528
Step #3501, epoch #350, avg. loss: 1.83631
Step #4001, epoch #400, avg. loss: 1.63447
Step #4501, epoch #450, avg. loss: 1.50723
Step #1, avg. loss: 820.59235
Step #501, epoch #50, avg. loss: 22.66042
Step #1001, epoch #100, avg. loss: 6.54875
Step #1501, epoch #150, avg. loss: 4.61562
Step #2001, epoch #200, avg. loss: 3.48994
Step #2501, epoch #250, avg. loss: 2.80458
Step #3001, epoch #300, avg. loss: 2.35560
Step #3501, epoch #350, avg. loss: 2.07383
Step #4001, epoch #400, avg. loss: 1.87293
Step #4501, epoch #450, avg. loss: 1.70909
Step #1, avg. loss: 698.36407
Step #501, epoch #50, avg. loss: 22.10735
Step #1001, epoch #100, avg. loss: 6.46612
Step #1501

TensorFlowEstimator(batch_size=100, class_weight=None,
          continue_training=False, early_stopping_rounds=None,
          keep_checkpoint_every_n_hours=10000, learning_rate=0.1,
          max_to_keep=5, model_fn=<function tanh_dnn at 0x1159d9488>,
          n_classes=0, num_cores=4, optimizer='SGD', steps=5000,
          tf_master='', tf_random_seed=42, verbose=1)